# DISCLAIMER
Copyright 2021 Google LLC. 

*This solution, including any related sample code or data, is made available on an “as is,” “as available,” and “with all faults” basis, solely for illustrative purposes, and without warranty or representation of any kind. This solution is experimental, unsupported and provided solely for your convenience. Your use of it is subject to your agreements with Google, as applicable, and may constitute a beta feature as defined under those agreements. To the extent that you make any data available to Google in connection with your use of the solution, you represent and warrant that you have all necessary and appropriate rights, consents and permissions to permit Google to use and process that data. By using any portion of this solution, you acknowledge, assume and accept all risks, known and unknown, associated with its usage, including with respect to your deployment of any portion of this solution in your systems, or usage in connection with your business, if at all.*

# Crystalvalue Demo: Predictive Customer LifeTime Value using Synthetic Data

This demo runs the Crystalvalue python library in a notebook using generated synthetic data. This notebook assumes that it is being run from within a [Google Cloud Platform AI Notebook](https://console.cloud.google.com/vertex-ai/notebooks/list/instances) with a Compute Engine default service account (the default setting when an AI Notebook is created) and with a standard Python 3 environment. For more details on the library please see the readme or for a more in-depth guide, including scheduling predictions, please see the `demo_with_real_data_notebook.ipynb`.

In [ ]:
import pandas as pd

from src import crystalvalue

In [ ]:
# Initiate the CrystalValue class with the relevant parameters.
pipeline = crystalvalue.CrystalValue(
  project_id='your_project_name',  # Enter your GCP Project name.
  dataset_id='a_dataset_name'  # The dataset will be created if it doesn't exist.
  )  

In [ ]:
# Create a synthetic dataset and load it to BigQuery.
data = pipeline.create_synthetic_data(table_name='synthetic_data')

In [ ]:
# Or create a transaction dataset from GA4 tables and load it to BigQuery. 
# Make sure the BQ client has access to the GA4 tables.
data = pipeline.preprocess_ga4_table(
    ga4_project_id='your_ga4_project_name',
    ga4_dataset_id='your_ga4_dataset_name',
    ga4_events_table_name='your_ga4_events_table_name'
)

In [ ]:
# Create summary statistics of the data and load it to Bigquery.
summary_statistics = pipeline.run_data_checks(transaction_table_name='synthetic_data')

In [ ]:
# Feature engineering for model training with test/train/validation split.
crystalvalue_train_data = pipeline.feature_engineer(
    transaction_table_name='synthetic_data')  

In [ ]:
# Train an AutoML model. 
model_object = pipeline.train_automl_model()

In [ ]:
# Deploy the model. 
model_object = pipeline.deploy_model()

In [ ]:
# Evaluate the model.
metrics = pipeline.evaluate_model()

In [ ]:
# Create features for prediction.
crystalvalue_predict_data = pipeline.feature_engineer(
    transaction_table_name='synthetic_data',  
    query_type='predict_query')

# Predict LTV for all customers.
predictions = pipeline.predict(
    input_table=crystalvalue_predict_data,
    destination_table='crystalvalue_predictions'  # The table will be created if it doesn't exist.
    )  

# Clean Up

To clean up tables created during this demo, delete the BigQuery tables that were created. All Vertex AI resources can be removed from the [Vertex AI console](https://console.cloud.google.com/vertex-ai). 

In [ ]:
pipeline.delete_table('synthetic_data')
pipeline.delete_table('crystalvalue_data_statistics')
pipeline.delete_table('crystalvalue_evaluation')
pipeline.delete_table('crystalvalue_train_data')
pipeline.delete_table('crystalvalue_predict_data')
pipeline.delete_table('crystalvalue_predictions')